In [50]:
!pip install stanza
import stanza
import sqlite3

ERROR: Operation cancelled by user


In [22]:
con = sqlite3.connect("/content/proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:  #some rows are NULL, we avoid them
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL")
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [10]:
with open("/content/ALL_TITLES_with-NULL.txt", "w", encoding="utf-8") as f: ##N: eso para tus ojos, no es necesario en tu workflow
  ##dado que tienes tu lista titles_list sobre la cual puedes trabajar (y aplicarle processing_1 definida más abajo)
  for elemento in titles_list:
    f.write(elemento + "\n")

In [14]:
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [processing_1(row, nlp_es) for row in titles_list]

processed_titles_es

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('A', 'a', 'ADP', 3, 'case'),
  ('el', 'el', 'DET', 3, 'det'),
  ('Sr.', 'sr.', 'PROPN', 7, 'obl'),
  ('D.', 'd.', 'PROPN', 3, 'flat'),
  ('R.', 'R.', 'PROPN', 3, 'flat'),
  ('P.', 'p.', 'PROPN', 3, 'flat'),
  ('Enviando', 'enviar', 'VERB', 0, 'root'),
  ('le', 'él', 'PRON', 7, 'obl:arg'),
  ('mis', 'mi', 'DET', 10, 'det'),
  ('obras', 'obra', 'NOUN', 7, 'obj')],
 [('La', 'el', 'DET', 2, 'det'),
  ('mujer', 'mujer', 'NOUN', 0, 'root'),
  ('piadosa', 'piadoso', 'ADJ', 2, 'amod'),
  ('(', '(', 'PUNCT', 5, 'punct'),
  ('Fragmentos', 'Fragmentos', 'PROPN', 2, 'appos'),
  (')', ')', 'PUNCT', 5, 'punct')],
 [('Desvarío', 'desvarío', 'NOUN', 0, 'root')],
 [('Soneto', 'soneto', 'PROPN', 0, 'root'),
  ('.', '.', 'PUNCT', 1, 'punct'),
  ('"', '"', 'PUNCT', 3, 'punct'),
  ('¡', '¡', 'PUNCT', 3, 'punct'),
  ('Gloria', 'gloria', 'PROPN', 0, 'root'),
  ('y', 'y', 'CCONJ', 5, 'cc'),
  ('pobreza', 'pobreza', 'NOUN', 3, 'conj'),
  ('!', '!', 'PUNCT', 3, 'punct'),
  ('El', 'el', 'DET', 3, 'det'),
  ('

In [15]:
with open("/content/WITH-NULL_ES_PROCESSING_PERTITLE.txt", "w", encoding="utf-8") as f:
  for element in processed_titles_es:
    f.write(str(element) + "\n")

In [24]:
import re
def find_delimit_subtit(text):
  find_subtit = re.search(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*', text)
  if find_subtit:
    return find_subtit.group(0)

  #cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  #return cleaned_text

subtit_ = [find_delimit_subtit(row) for row in titles_list]

with open("/content/testing.txt", "w", encoding="utf-8") as f:
  for element in subtit_:
    f.write(str(element) + "\n")

In [23]:
#Delimitation of titles (later recognizing them)
import re
def find_delimit_subtit(text):
  subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r' | ', text)
  return subtitle_

encontrar_subtit = [find_delimit_subtit(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('"', '| ')
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')

print(encontrar_subtit[249])
print(encontrar_subtit[207])


Inauguración del teatro de D. José Piquer-Oda | Alzad vuestras tumbas [...]| 
Matrimonios por anuncios | Los baños de Biarritz | Hace pocos días [...]"


In [6]:
with open("/content/subtit_delimitados.txt", "w", encoding="utf-8") as f:
  for element in encontrar_subtit:
    f.write(str(element) + "\n")

In [41]:
import re

def clean_per_title(title):
  cleaned_text = re.sub(
    r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*',
    ' ',
    title)
  cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  return cleaned_text.strip()

cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

print(cleaned_rows)


['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa', 'Desvarío', 'Soneto. ¡Gloria y pobreza! El inmortal Quintana...', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna.', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna.', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna.', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona.', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Alteza Real el Príncipe de Asturias', 'Margarita. Novela bretona.', 'El otoño

In [42]:
with open("/content/raw_subtit_tit.txt", "w", encoding="utf-8") as f:
  for element in cleaned_rows:
    f.write(str(element) + "\n")

In [48]:
for row in cleaned_rows:
  if " | " in row:
    cleaned_rows[i] = row.split(" | ")

for row in cleaned_rows:
  print(row, "\n")

Al Sr. D. R. P. Enviándole mis obras 

La mujer piadosa 

Desvarío 

Soneto. ¡Gloria y pobreza! El inmortal Quintana... 

Balada. La despedida de una hija 

Un viaje a la Luna. Prólogo: La pesadilla de Mocquet 

En el álbum de la señora M… 

La primavera 

La perejilera 

Un viaje a la Luna. 

Al señor Don B. M. 

A orillas del Manzanares. La súplica 

Un viaje a la Luna. 

Fragmentos de la Rosa de Alejandra 

Gente morena 

Un viaje a la Luna. 

Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo 

A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños 

Costumbres del siglo XIX. I 

Margarita. Novela bretona 

Los escondidos y la tapada. Fragmentos 

A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum 

Costumbres del siglo XIX.II 

Margarita. Novela bretona. 

A la niña Margarita Aurora de Wilson y Serrano 

A María 

A su Alteza Real el Príncipe de Asturias 

Margarita. Novela bretona. 

El otoño 

Soneto. Aún eco tienen en e

In [ ]:
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [processing_1(row, nlp_es) for row in titles_list]

processed_titles_es

In [ ]:
# Before applying NER, first cleaning of each title
# (only specific symbols that affect NER, such us "[...]", "-", ".") %%

import re
def clean_text(text):
  cleaned_text = re.search(r'\s*"\s*|\s*-\s*|\s*\[\.\.\.\]\s*', ' ', text)
  cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
  return cleaned_text

cleaned_rows = [clean_text(row) for row in titles_list]

print(cleaned_rows)

with open("content/CLEAN_ALL_TITLES.txt", "w", encoding="utf-8") as f:
    for element in cleaned_rows:
        f.write(str(element) + "\n")

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. ¡Gloria y pobreza! El inmortal Quintana...', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Alte

In [ ]:
with open("content/CLEAN_ALL_TITLES.txt", "w", encoding="utf-8") as f:
    for element in cleaned_rows:
        f.write(str(element) + "\n")

In [ ]:
#let's iterate on cleaned_rows through NER function

def ner1(text, nlp_model):
  doc = nlp_model(text)
  return [(ent.text, ent.type) for ent in doc.entities]

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner")

cleaned_entities_ES = [ner1(title, nlp_es) for title in cleaned_rows]

print(cleaned_entities_ES)

with open("content/CLEANED_ENTITIES_ES.txt", "w", encoding="utf-8") as f:
    for element in cleaned_entities_ES:
        f.write(str(element) + "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('Sr. D. R. P.', 'MISC')], [('Fragmentos', 'MISC')], [('Desvarío', 'MISC')], [('Soneto', 'PER'), ('Quintana', 'PER')], [('Balada', 'PER')], [('Luna', 'MISC'), ('Mocquet', 'PER')], [('señora M…', 'PER')], [], [], [('Luna', 'MISC')], [('Don B. M.', 'PER')], [('Manzanares', 'LOC')], [('Luna', 'MISC')], [('de la Rosa de Alejandra', 'PER')], [], [('Luna', 'MISC')], [('Sr. D. Agustín Esteban Collantes', 'MISC')], [('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')], [('I', 'PER')], [('Margarita', 'PER')], [], [('Señora doña M. Del Pilar Sinués de Marco', 'PER')], [('II', 'MISC')], [('Margarita', 'PER')], [('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')], [('María', 'PER')], [('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')], [('Margarita', 'PER')], [], [('Soneto', 'PER')], [('SS. MM. II', 'PER')], [('Princesa', 'PER'), ('Inglaterra', 'LOC')], [('Poesía', 'MISC'), ('Oda', 'PER')], [('Alejandro Dumas', 'PER')], [('Soneto', 'PER')], [], [('SS. AA', 'ORG'), ('príncipe M

In [ ]:
# and now, we also want to process (tok, lemma, mkw, pos) the NER string,
# without applying depparse %%
def processing_2(text, nlp_model):
  doc = nlp_model(text)
  return [(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words]

clean_processed_entities_ES = []

for entity_list in cleaned_entities_ES:
  row_result = []
  for entity in entity_list:
    text = entity[0]
    tok = processing_2(text, nlp_es)
    row_result.append([entity, tok])

  clean_processed_entities_ES.append(row_result) #IMPORTANT TO NOT INDENT THIS LINE, OTHERWISE IT DOES NOT KEEP EMPTY ROWS(THAT IS, WITHOUT ENTITY)

with open("content/CLEAN_MERGED_TITLE_NER.txt", "w", encoding="utf-8") as f:
    for element in clean_processed_entities_ES:
        f.write(str(element) + "\n")

clean_processed_entities_ES[:10]

[[[('Sr. D. R. P.', 'MISC'),
   [('Sr', 'sr', 'PROPN'),
    ('.', '.', 'PUNCT'),
    ('D.', 'd.', 'PROPN'),
    ('R.', 'R.', 'PROPN'),
    ('P.', 'p.', 'PROPN')]]],
 [[('Fragmentos', 'MISC'), [('Fragmentos', 'fragmentos', 'NOUN')]]],
 [[('Desvarío', 'MISC'), [('Desvarío', 'desvarío', 'NOUN')]]],
 [[('Soneto', 'PER'), [('Soneto', 'soneto', 'NOUN')]],
  [('Quintana', 'PER'), [('Quintana', 'Quintana', 'PROPN')]]],
 [[('Balada', 'PER'), [('Balada', 'Balada', 'PROPN')]]],
 [[('Luna', 'MISC'), [('Luna', 'luna', 'PROPN')]],
  [('Mocquet', 'PER'), [('Mocquet', 'Mocquet', 'PROPN')]]],
 [[('señora M…', 'PER'),
   [('señora', 'señora', 'NOUN'), ('M…', 'm…', 'PROPN')]]],
 [],
 [],
 [[('Luna', 'MISC'), [('Luna', 'luna', 'PROPN')]]]]

In [ ]:
# let's locate the entire entity and its processing [[(),[()]]] where their
# strings are located within each title %%
import re
#clean_processed_entities_ES
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(clean_processed_entities_ES):
  for componentes in titulo:
    for tuple_item in componentes:
      entity_str = tuple_item[0][0]
      if entity_str in cleaned_rows_copy[i]:
        replacement_str = f"{tuple_item}"
        cleaned_rows_copy[i] = re.sub(rf'\b{entity_str}\b', replacement_str, cleaned_rows_copy[i])

print("\n".join(cleaned_rows_copy))

Al [('Sr', 'sr', 'PROPN'), ('.', '.', 'PUNCT'), ('D.', 'd.', 'PROPN'), ('R.', 'R.', 'PROPN'), ('P.', 'p.', 'PROPN')]. D. R. P. Enviándole mis obras
La mujer piadosa ([('Fragmentos', 'fragmentos', 'NOUN')])
[('Desvarío', 'desvarío', 'NOUN')]
[('Soneto', 'soneto', 'NOUN')]. ¡Gloria y pobreza! El inmortal [('Quintana', 'Quintana', 'PROPN')]...
[('Balada', 'Balada', 'PROPN')]. La despedida de una hija
Un viaje a la [('Luna', 'luna', 'PROPN')]. Prólogo: La pesadilla de [('Mocquet', 'Mocquet', 'PROPN')]
En el álbum de la [('señora', 'señora', 'NOUN'), ('M…', 'm…', 'PROPN')] M…
La primavera
La perejilera
Un viaje a la [('Luna', 'luna', 'PROPN')]. (Continuación)
Al señor [('Don', 'Don', 'PROPN'), ('B.', 'B.', 'PROPN'), ('M.', 'M.', 'PROPN')] B. M.
A orillas del [('Manzanares', 'manzanares', 'PROPN')]. La súplica
Un viaje a la [('Luna', 'luna', 'PROPN')]. (Continuación)
Fragmentos [('de', 'de', 'ADP'), ('la', 'el', 'DET'), ('Rosa', 'Rosa', 'PROPN'), ('de', 'de', 'ADP'), ('Alejandra', 'Alejandra